In [ ]:
import sys
sys.path.insert(1, '/ceph/users/apua/projects/v2_caudate_gene_vae_394/code/_h/disentangling-vae')

import pandas as pd
import torch
from disvae.utils.modelIO import load_model, load_metadata
from utils.datasets import get_dataloaders

In [ ]:
dfz = pd.read_csv("../../../../input/prep_data_cv/_m/Fold-alldata/X_train.csv", index_col=0)

In [ ]:
model_dir = "../../_m/results/factor_caudategenes_fc5_fold_alldata/"
model = load_model(model_dir, filename = "model.pt")
metadata = load_metadata(model_dir)

In [ ]:
dl = get_dataloaders("caudategenes", train_or_test='train', fold='alldata', shuffle=False, batch_size=1)


In [ ]:
def latent_mu(dl):
    model.eval()
    for data, _ in dl:
        mu, logvar = model.encoder(data.to("cuda"))
        
        yield mu.cpu().detach().numpy()[0]

df_mu = pd.DataFrame.from_records((latent_mu(dl)), index=dfz.index)
df_mu.columns = ["mu" + str(x) for x  in df_mu.columns]

In [ ]:
def latent_lv(dl):
    model.eval()
    i = 0
    for data, _ in dl:
        mu, logvar = model.encoder(data.to("cuda"))
        
        yield logvar.cpu().detach().numpy()[0]

df_lv = pd.DataFrame.from_records((latent_lv(dl)), index=dfz.index)
df_lv.columns = ["lv" + str(x) for x  in df_lv.columns]

In [ ]:
dfo = pd.concat([df_mu, df_lv], axis=1)
dfo.to_csv('latent_variables.csv')

In [ ]:
df_mu.head(6)

In [ ]:
from plotnine import *

In [ ]:
ggplot(df_mu, aes(x='mu4', y='mu5')) + geom_point(alpha=.01)

In [ ]:
import umap

In [ ]:
embedding = umap.UMAP(n_neighbors=3, min_dist=0.02, metric='euclidean').fit_transform(df_mu.values)


In [ ]:
dfe = pd.DataFrame(embedding, columns =['dim1', 'dim2'], index=df_mu.index)


In [ ]:
ggplot(dfe, aes(x='dim1', y='dim2')) + geom_point(alpha=.01)

In [ ]:
dfe.head()

In [ ]:
dfe.shape

In [ ]:
#dfe.to_csv('umap_embedding.csv')

In [ ]:
dfp = dfo.copy()
dfp['mean_expr'] = dfz.mean(axis=1)


In [ ]:
for var in dfo.columns:
    p = ggplot(dfp, aes(x='mean_expr', y=var)) + geom_point(alpha=.01)
    print(p)

In [ ]:
1